## Import

In [1]:
import os
import pandas as pd
import math, random

def import_csv(filepath, sep="\t"):
    with open(filepath, "r") as fh:
        data = {}
        
        for line in fh.readlines():
            line = line.strip().split(sep)
            
            if line[0] != "" and len(line) > 1:
                data[line[0]] = line[1]
                
        return data
    
organism_paths =  ["Organisms/" + file for file in os.listdir("Organisms")]
cell_line_paths = ["Cell Lines/" + file for file in os.listdir("Cell Lines")]

seq_data = import_csv("temp_data.tsv")
all_data = []

for file_path in organism_paths + cell_line_paths:
    excel_data = pd.read_excel(file_path)
    extract = excel_data[["UNIPROT_ID", "Tm_(C)"]]
    
    for i in range(len(extract)):
        if str(type(extract.at[i, "Tm_(C)"])) == "<class \'str\'>":
            extract.at[i, "Tm_(C)"] = float(extract.at[i, "Tm_(C)"].split(" ")[0])
        if str(type(extract.at[i, "UNIPROT_ID"])) != "<class \'str\'>":
            extract.at[i, "UNIPROT_ID"] = str(extract.at[i, "UNIPROT_ID"])
            
        #check for NaN (in Tm?)
        if math.isnan(extract.at[i, "Tm_(C)"]):
            continue
        
        #check that Uniprot ID is in seq_data
        if extract.at[i, "UNIPROT_ID"] not in seq_data.keys():
            continue
            
        all_data.append([extract.at[i, "Tm_(C)"], seq_data[extract.at[i, "UNIPROT_ID"]]])
        
#shuffle data
random.seed(42)
new_indices = random.sample(list(range(len(all_data))), len(all_data))
all_data = [all_data[i] for i in new_indices]
all_data = [record for record in all_data if len(record[1]) != 1]

## RF model

In [3]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

hydropathy_index = {
    "A":1.8,
    "R":-4.5,
    "N":-3.5,
    "D":-3.5,
    "C":2.5,
    "E":-3.5,
    "Q":-3.5,
    "G":-0.4,
    "H":-3.2,
    "I":4.5,
    "L":3.8,
    "K":-3.9,
    "M":1.9,
    "F":2.8,
    "P":-1.6,
    "S":-0.8,
    "T":-0.7,
    "W":-0.9,
    "Y":-1.3,
    "V":4.2
}

def calc_gravy(seq):
    avg = 0.0
    for char in seq:
        try:
            hindex = hydropathy_index[char]
            avg += hindex
        except:
            pass #ignore uncommon amino acids
        
    return avg/len(seq)

def generate_hydropathy_seq(seq):
    hydropathy_seq = []
    
    for char in seq:
        try:
            hydropathy_seq.append(hydropathy_index[char])
        except:
            pass #ignore uncommon amino acids
        
    return hydropathy_seq

#I believe this may not be quite perfect?
def linear_sequence_scaling(seq, to_size):
    if len(seq) == to_size:
        return seq
    
    elif len(seq) == 1:
        return [0.0 for i in range(to_size)]

    elif len(seq) < to_size: #scale up
        new_seq = []
        blank_elements = to_size - len(seq)
        insertion_size = to_size // (len(seq)-1)

        for i in range(len(seq)-1):
            new_seq.append(seq[i])

            element_count = min(insertion_size, blank_elements)
            for j in range(element_count):
                weighted_val = ((1-(j/float(element_count))) * seq[i]) + ((j/float(element_count)) * seq[i+1])
                new_seq.append(weighted_val)

            blank_elements -= insertion_size

        new_seq.append(seq[-1])
        
        return new_seq

    elif len(seq) > to_size: #scale down
        window = len(seq) - to_size
        return [average(seq[i:i+window]) for i in range(len(seq) - window)]

def average(seq):
    return sum(seq) / float(len(seq))

#generate proxy variable(s) and shape data
#x = [[calc_gravy(record[1])] for record in all_data] #0.2
x = [linear_sequence_scaling(generate_hydropathy_seq(record[1]), 100) for record in all_data] #0.32
y = [record[0] for record in all_data]

#test-train split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

#fit random forest
Tm_rf = RandomForestRegressor(n_estimators = 100)
Tm_rf.fit(x_train, y_train)

#run random forest on test set
print(Tm_rf.score(x_test, y_test))

0.3240783256511316


In [ ]:
length_set = set()

for seq in x:
    length_set.add(len(seq))
    
print(length_set)